In [14]:
import requests
import copy
import csv
import time
import json
import uuid
import pprint
import asyncio
from typing import List
from urllib import parse
from urllib.parse import quote
import inspect
import datetime

In [15]:
class magic_fstring_function:
    def __init__(self, payload):
        self.payload = payload
        self.cached = None
    def __str__(self):
        if self.cached is None:
            vars = inspect.currentframe().f_back.f_globals.copy()
            vars.update(inspect.currentframe().f_back.f_locals)
            self.cached = self.payload.format(**vars)
        return self.cached

In [16]:
uuid = str(uuid.uuid4())
item_list_api = "https://apis.zigbang.com/v2/items"
describe_list_api = item_list_api + '/list'
item_describe_api = "https://apis.zigbang.com/v3/items?item_ids={item_id}&detail=true"
item_view_url = "https://zigbang.com/home/oneroom/items/{item_id}"

referer = "https://zigbang.com/home/oneroom/subways/414/items"
        
headers = {
    'Host':'apis.zigbang.com',
    'Connection':'keep-alive',
    'Pragma':'no-cache',
    'Cache-Control':'no-cache',
    'Accept':'application/json, text/plain, */*',
    'Origin':'https://zigbang.com',
    'User-Agent':'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Mobile Safari/537.36',
    'DNT':'1',
    'Sec-Fetch-Site':'same-site',
    'Sec-Fetch-Mode':'cors',
    'Referer':'https://zigbang.com/home/oneroom/subways/414/items',
    'Accept-Encoding':'gzip, deflate, br',
    'Accept-Language':'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
}

items = {}

In [17]:
def export_items(items):
    fieldnames = [
        'address1',
        'address2',
        'address3',
        'agent_address1',
        'agent_email',
        'agent_lat',
        'agent_lng',
        'agent_mobile',
        'agent_name',
        'agent_phone',
        'building_type',
        'deposit',
        'description',
        'elevator',
        'floor',
        'floor_all',
        'images',
        'local1',
        'local2',
        'local3',
        'manage_cost',
        'manage_cost_inc',
        'movein_date',
        'near_subways',
        'options',
        'parking',
        'pets_text',
        'random_location',
        'room_direction_text',
        'room_type',
        'sales_type',
        'service_type',
        'size',
        'size_m2',
        'title',
        'updated_at2',
        'user_intro',
        'user_name',
        'url'
    ]
    
    filtered_item = [] 
    maximum_updated = datetime.timedelta(days=7)
    

    for item in items:
        item = item['item']

        item_updated = datetime.datetime.strptime(item['updated_at2'], '%Y-%m-%d')
        if maximum_updated < datetime.datetime.now() - item_updated:
            continue
        item_id = item['id']
        url = magic_fstring_function(item_view_url)
        item['url'] = str(url)
        new_item = {}
        for fieldname in fieldnames:
            new_item[fieldname] = item[fieldname]
        filtered_item.append(new_item)
    return filtered_item

In [18]:
def describe_room_list(items):
    item_ids = list(items.keys())
    max_idx = int(len(item_ids) / 30)
    if len(item_ids) % 30 != 0:
        max_idx += 1
    
    items = []
    headers['Accept'] = 'application/json'
    headers['Referer'] = 'https://zigbang.com/home/oneroom/subways/37/items'
    
    for i in range(max_idx):
        start_idx = i * 30
        end_idx = min((i+1)*30, len(item_ids))
        sub_ids = item_ids[start_idx:end_idx-1]
        item_id = str(sub_ids).replace(' ', '').replace('\'','')
        api = magic_fstring_function(item_describe_api)
        g = api
        resp = requests.get(
            url=str(api),
            headers=headers,
            timeout=5,
        )
        resp.encoding = 'utf-8'
        result = resp.json()
        resp.close()
        items += result['items']
        time.sleep(1)
        
    return items

In [19]:
def get_room_list(
    items,
    domain: str = "zigbang",
    floor_in: str = "ground",
    geohash: str = "wyd",
    rent_gteq: int = 0,
    service_type_eq: str = "원룸",
):
    resp = requests.get(
        url=item_list_api,
        params={
            "domain": domain,
            "floor_in": floor_in,
            "geohash": geohash,
            "rent_gteq": rent_gteq,
            "service_type_eq": service_type_eq,
        },
        headers=headers,
        timeout=5,
    )
    new_items = json.loads(resp.content.decode('utf-8'))
    resp.close()

    for section in new_items['sections']:
        for item in section['item_ids']:
            items[str(item)] = {}

    return items

In [20]:
items = {}

In [21]:
geos_seoul = [
    'wydm'
] 

for geo in geos_seoul:
    items = get_room_list(items, geohash=geo)
    time.sleep(1)

In [22]:
from itertools import islice

sliceItem = dict(islice(items.items(), 3000))

len(sliceItem)

3000

In [23]:
print(sliceItem)

{'29072852': {}, '28975787': {}, '29254502': {}, '28808477': {}, '29008227': {}, '29273998': {}, '29196029': {}, '29136337': {}, '29278932': {}, '29114919': {}, '28838799': {}, '29254696': {}, '28962012': {}, '29234970': {}, '29269980': {}, '29271171': {}, '29265612': {}, '29277703': {}, '29028655': {}, '29212503': {}, '29270800': {}, '29031488': {}, '29113988': {}, '29279444': {}, '29000267': {}, '29186621': {}, '29132845': {}, '28882531': {}, '29216076': {}, '29181807': {}, '29256035': {}, '29193457': {}, '29234677': {}, '29266374': {}, '29268008': {}, '29274248': {}, '29148097': {}, '29009268': {}, '28972260': {}, '29253009': {}, '29257696': {}, '29181333': {}, '29158321': {}, '29263151': {}, '29255723': {}, '28996729': {}, '29098632': {}, '29005966': {}, '29275405': {}, '29253843': {}, '29063944': {}, '29036911': {}, '29190021': {}, '29209845': {}, '29279917': {}, '29274548': {}, '29137091': {}, '29167125': {}, '29255398': {}, '29273443': {}, '29106249': {}, '29264653': {}, '291795

In [24]:
result = describe_room_list(sliceItem)

In [25]:
len(result)

2900

In [26]:
pp = pprint.PrettyPrinter(indent=2)
pp.pprint(result[0]['item'])

{ '_floor': '5층',
  'address1': '서울시 광진구 화양동',
  'address2': '5-18',
  'address3': None,
  'agent_address1': '서울특별시 광진구 화양동 15-53',
  'agent_comment': '',
  'agent_email': 'tnsqhr314@naver.com',
  'agent_lat': 37.54434204101561,
  'agent_lng': 127.069938659668,
  'agent_local1': '서울시',
  'agent_local2': '광진구',
  'agent_mobile': '010-3708-0092',
  'agent_name': '로얄부동산공인중개사사무소(강순복)',
  'agent_no': 46096,
  'agent_phone': '02-462-3003',
  'bjd_code': '',
  'bonbun_code': '5',
  'bubun_code': '18',
  'building': None,
  'building_type': '다세대건물',
  'contract': '서울특별시',
  'deposit': 500,
  'description': '✔ Check Point \r\n'
                 '⬜ 금액 : 500/60\r\n'
                 '⬜ 구조 : 원룸건물/CCTV/이중보안/도어락/오픈형\r\n'
                 '⬜ 입주 : 즉시입주가능\r\n'
                 '⬜ 옵션 : 세탁기,냉장고,에어컨,쿡탑,신발장 등\r\n'
                 '⬜ 편의시설 : GS수퍼,파리바게트,건대병원,편의점,약국,재래시장\r\n'
                 '\r\n'
                 '\r\n'
                 '💗 상세정보\r\n'
                 '\r\n'
                 '신축 2년차 건대역 초역세권

In [27]:
filtered_result = export_items(result)

In [28]:
len(filtered_result)

1858

In [29]:
def get_string_image(item):
    image = ''
    
    for list in item:
        image += list['url'] + '?w=400&h=300&q=70&a=1,'
        
    return image[:-1]

In [30]:
def get_full_address(item):
    addr = item['address1']
    addr += ' '
    addr += item['address2'] or ''
    addr += ' '
    addr += item['address3'] or ''
    return addr

In [31]:
def get_location_lat(item):
    locationList = item['random_location'].split(',')

    lat = locationList[0][0:9]
    
    return lat

def get_location_lng(item):
    locationList = item['random_location'].split(',')

    lng = locationList[1][0:10]
    
    return lng

In [32]:
def get_agent_lat(item):
    agent_lat = str(item['agent_lat'])[0:9]
    
    return agent_lat

def get_agent_lng(item):
    agent_lng = str(item['agent_lng'])[0:10]
    
    return agent_lng

In [33]:
geo_based_dict = {}

In [34]:
for item in filtered_result:
    full_addr = get_full_address(item)
    item['address'] = full_addr
    if full_addr not in geo_based_dict:
        geo_based_dict[full_addr] = []
    geo_based_dict[full_addr].append(item)

    item['lat'] = get_location_lat(item)
    item['lng'] = get_location_lng(item)
    item['image'] = get_string_image(item['images'])
    item['agent_lat'] = get_agent_lat(item)
    item['agent_lng'] = get_agent_lng(item)   

In [36]:
first_key = next(iter(geo_based_dict))
pp.pprint(geo_based_dict[first_key][0])

{ 'address': '서울시 서초구 잠원동 10-18 ',
  'address1': '서울시 서초구 잠원동',
  'address2': '10-18',
  'address3': None,
  'agent_address1': '서울특별시 강남구 역삼로17길 9 101호(역삼동)',
  'agent_email': '0202jane@naver.com',
  'agent_lat': '37.495520',
  'agent_lng': '127.035802',
  'agent_mobile': '010-9579-2680',
  'agent_name': '에이스하우징공인중개사사무소(김미자)',
  'agent_phone': '010-3729-2680',
  'building_type': '다세대건물',
  'deposit': 2000,
  'description': '📢 입지조건\n'
                 '   🅰 대중교통 이용에 편리한 위치 (지하철, 버스등)\n'
                 '   🅰 안전을 대비한 인근 CCTV 배치\n'
                 '\n'
                 '📢 주변환경\n'
                 '   🅰 조용한생활이 가능한 주택가에 위치\n'
                 '   🅰 편리한 주변 편의시설\n'
                 '         ( 편의점, 마트, 약국, 미용실등 )\n'
                 '\n'
                 '📢 특징\n'
                 '   🅰 꾸준하고 꼼꼼하게 관리한 컨디션좋은 원룸\n'
                 '   🅰 여러매물중 가성비 훌륭한 원룸\n'
                 '   🅰 상태좋은 옵션들과 채광좋고 환기잘되는 원룸\n'
                 '\n'
                 '📢 주차\n'
                 '   🅰 주차는 1대 가능합니다\n'
  

In [37]:
double_list = []
for addr in geo_based_dict.keys():
    double_list.append(geo_based_dict[addr][0])
double_list.sort(key=lambda x: x['size'], reverse=True)

In [38]:
with open('oneroom.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = [
        'address',
        'agent_address1',
        'agent_email',
        'agent_lat',
        'agent_lng',
        'agent_mobile',
        'agent_name',
        'agent_phone',
        'building_type',
        'deposit',
        'description',
        'elevator',
        'floor',
        'floor_all',
        'image',
        'lat',
        'lng',
        'local1',
        'local2',
        'local3',
        'manage_cost',
        'manage_cost_inc',
        'movein_date',
        'near_subways',
        'options',
        'parking',
        'pets_text',
        'room_direction_text',
        'room_type',
        'sales_type',
        'service_type',
        'size',
        'size_m2',
        'title',
        'updated_at2',
        'user_intro',
        'user_name',
        'url'
    ]
    writer = csv.DictWriter(csvfile, extrasaction='ignore', fieldnames=fieldnames)
    writer.writeheader()
    for item in double_list:
        writer.writerow(item)

In [1]:
import pymysql
import csv
conn = pymysql.connect(host='localhost', user='khweb', password='khWeb@123', db='mybang_db', charset='utf8mb4')
curs = conn.cursor()

sql = """insert into oneroom (address, agent_address, agent_email, agent_lat, agent_lng,
                            agent_mobile, agent_name, agent_phone, building_type, deposit, 
                            description, elevator, floor, floor_all, image,
                            lat, lng, local1, local2, local3,
                            manage_cost, manage_cost_inc, movein_date, near_subways, options,
                            parking, pets, room_direction, room_type, sales_type,
                            service_type, size, size_m2, title, updated_at,
                            user_intro, user_name, url)
                            values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,
                                    %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,
                                    %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,
                                    %s,%s,%s,%s,%s,%s,%s,%s)"""

f = open('oneroom.csv', 'r', encoding='utf-8')
rd = csv.reader(f)

next(rd)

for line in rd:
    curs.execute(sql, (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], 
                       line[11], line[12], line[13], line[14], line[15], line[16], line[17], line[18], line[19], line[20], 
                       line[21], line[22], line[23], line[24], line[25], line[26], line[27], line[28], line[29], line[30], 
                       line[31], line[32], line[33], line[34], line[35], line[36], line[37]))
conn.commit()
conn.close()
f.close()